In [3]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

SyntaxError: invalid syntax (147025888.py, line 3)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
from collections import Counter
from skimage import io, transform
from torch.nn.utils.rnn import pack_padded_sequence
import matplotlib.pyplot as plt # for plotting
import numpy as np
from time import time
import collections
import pickle
import os
import gensim
import nltk
import re

In [2]:
class Rescale(object):
    """Rescale the image in a sample to a given size.
    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, image):
        h, w = image.shape[:2]
        #print("TA RESCALE INPUT", image.shape)
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)
        img = transform.resize(image, (new_h, new_w))
        #print("TA RESCALE OUTPUT", image.shape)
        return img

In [3]:
os.getcwd()

'/Users/pratyushsaini/Documents/Semester 5/COL774/Assignment-4'

In [4]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, image):
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return torch.tensor(image)

In [5]:
IMAGE_RESIZE = (256, 256)
device = "cuda" if torch.cuda.is_available() else "cpu"
img_transform = transforms.Compose([Rescale(IMAGE_RESIZE), ToTensor(), transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) ])
print("Current device set to {}".format(device))
DIR = '/Users/pratyushsaini/Documents/Semester 5/COL774/Assignment-4/'

Current device set to cpu


In [6]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize
phase = "Train"

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pratyushsaini/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/pratyushsaini/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
class CaptionsPreprocessing:
    """Preprocess the captions, generate vocabulary and convert words to tensor tokens

    Args:
        captions_file_path (string): captions tsv file path
    """
    def __init__(self, captions_file_path):
        self.captions_file_path = captions_file_path
        self.raw_captions_dict = self.read_raw_captions()
        self.captions_dict = self.process_captions()
        self.vocab = self.generate_vocabulary()
    def read_raw_captions(self):
        # Dictionary with raw captions list keyed by image ids (integers)
        captions_dict = {}
        with open(self.captions_file_path, 'r', encoding='utf-8') as f:
            for img_caption_line in f.readlines():
                img_captions = img_caption_line.strip().split('\t')
                image_path = DIR + img_captions[0]
                if os.path.exists(image_path):
                    captions_dict[img_captions[0]] = img_captions[1]
        return captions_dict

    def process_captions(self):
        """
        Use this function to generate dictionary and other preprocessing on captions
        """

        raw_captions_dict = self.raw_captions_dict

        # Do the preprocessing here
        # Can remove the stopwords and gibberish in the caption
        stop_words = stopwords.words('english')
        punctuation = list(string.punctuation)

        for key, value in raw_captions_dict.items():
            cleaned_caption = re.sub('[^A-Za-z0-9]+', ' ', value) #Extra space removal
            tokens = word_tokenize(cleaned_caption)
            cleaned_tokens = [token for token in tokens if token not in stop_words and token not in punctuation] # Remove stopwords and punctuation
            cleaned_caption = "[START] " + " ".join(cleaned_tokens) + " [END]"
            raw_captions_dict[key] = cleaned_caption        

        captions_dict = raw_captions_dict

        return captions_dict

    def generate_vocabulary(self):
        """
        Use this function to generate dictionary and other preprocessing on captions
        """

        captions_dict = self.captions_dict
        vocabulary = {}
        max_caption = 0
        idx = 1
        index_to_word = {}
        for key, value in captions_dict.items():
            val = value.split()
            max_caption = max(max_caption, len(val))

            for i in val:
                if i not in vocabulary.keys():
                    vocabulary[i] = idx
                    index_to_word[idx] = i
                    idx+=1
        self.max_caption = max_caption
        index_to_word[0] = "NIL"
        self.index_to_word = index_to_word
        # Generate the vocabulary
        print("Size of Vocabulary = {}".format(len(vocabulary)))
        return vocabulary


    def get_captions(self, tensor_tokens):
        caption = [self.index_to_word[int(x)] for x in tensor_tokens]
        return " ".join(caption)

    def captions_transform(self, img_caption):
        """
        Use this function to generate tensor tokens for the text captions
        Args:
            img_caption_list: List of captions for a particular image
        """
        vocab = self.vocab

        caption = img_caption.split(" ")
        
        
        # print(img_caption, caption)

        caption_mapped = np.zeros(self.max_caption)
        for i in range(len(caption)):
            try: caption_mapped[i] = self.vocab[caption[i]]
            except: print(img_caption, caption, i)

        # caption_mapped = np.zeros((self.max_caption, len(self.vocab)))
        # for i in range(len(caption)):
        #     val = np.zeros(len(self.vocab))
        #     val[self.vocab[caption[i]]] = 1
        #     caption_mapped[i,:] = val 

        #captions_mapped = np.argmax(captions_mapped, axis = 1)
        
        return torch.LongTensor(caption_mapped)

# Set the captions tsv file path

#CAPTIONS_FILE_PATH = '/content/drive/MyDrive/data/train_text.tsv'
CAPTIONS_FILE_PATH = '/Users/pratyushsaini/Documents/Semester 5/COL774/Assignment-4/Train_text.tsv'
captions_preprocessing_obj = CaptionsPreprocessing(CAPTIONS_FILE_PATH)
embedding_dim = 256

Size of Vocabulary = 7356


In [8]:
ls

Grishkin_Sene.pdf
Train_text.tsv
Vinyals_Show_and_Tell_2015_CVPR_paper.pdf
q1.ipynb
q1_new.ipynb
q2.ipynb
starter_code.ipynb
test_data/
test_data.zip
train_data/
train_data.zip


In [9]:
class ImageCaptionsDataset(Dataset):

    def __init__(self, img_dir, captions_dict, img_transform=None, captions_transform=None):
        """
        Args:
            img_dir (string): Directory with all the images.
            captions_dict: Dictionary with captions list keyed by image paths (strings)
            img_transform (callable, optional): Optional transform to be applied
                on the image sample.

            captions_transform: (callable, optional): Optional transform to be applied
                on the caption sample (list).
        """
        self.img_dir = img_dir
        self.captions_dict = captions_dict
        self.img_transform = img_transform
        self.captions_transform = captions_transform

        self.image_ids = list(captions_dict.keys())

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_name = self.image_ids[idx]
        image = io.imread(img_name)
        captions = self.captions_dict[img_name]

        if self.img_transform:
            image = self.img_transform(image)

        if self.captions_transform:
            captions = self.captions_transform(captions)

        sample = {'image': image, 'captions': captions}

        return sample

In [10]:
def collate_fn(batch):
  res = {}

  res['image'] = [sample['image'].unsqueeze(0) for sample in batch] 
  res['image'] = torch.cat((res['image']), dim=0)

  res['captions'] = [sample['captions'] for sample in batch]
  res['captions'] = torch.nn.utils.rnn.pad_sequence(res['captions'], batch_first=True)

  return res

In [11]:
#ENCODER
class Encoder(nn.Module):
    def __init__(self, embed_dim, trainCNN = False):
        super(Encoder, self).__init__()
        # resnet50 = models.resnet50(pretrained=True, progress=True)        
        # self.resnet50 = resnet50
        # for param in self.resnet50.parameters():
        #     param.requires_grad = False
        # self.fc = nn.Linear(in_features=self.resnet50.fc.in_features, out_features=embed_dim, bias = True)
        # layers = list(resnet50.children())[:-1]
        # self.resnet50 = nn.Sequential(*layers)
        # self.relu = nn.LeakyReLU()
        # print("resnet50 Loaded Successfully..!")
        self.trainCNN = trainCNN
        self.inception = torchvision.models.inception_v3(pretrained=True, aux_logits = False)
        self.inception.fc = nn.Linear(in_features=self.inception.fc.in_features, out_features=embed_dim, bias = True)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        #print("Forward feeding")
        features = self.inception(x)
        #print("Resnet module op", x.shape)
        for name, param in self.inception.named_parameters():
            if "fc.weight" in name or "fc.bias" in name:
               param.requires_grad = True
            else:
                param.requires_grad = self.trainCNN
        return self.dropout(self.relu(features))

        

class Decoder(nn.Module):
    def __init__(self, embed_dim, lstm_hidden_size, vocab_size, lstm_layers=1):
        super(Decoder, self).__init__()
        self.lstm_hidden_size = lstm_hidden_size
        self.vocab_size = len(captions_preprocessing_obj.vocab)
        print("VOCAB SIZE = ", self.vocab_size)
        
        self.lstm = nn.LSTM(input_size = embed_dim, hidden_size = lstm_hidden_size,
                            num_layers = lstm_layers, batch_first = True)
        #self.attention = AttentionBlock(embed_dim, lstm_hidden_size, self.vocab_size)
        self.linear = nn.Linear(lstm_hidden_size, self.vocab_size)        
        #self.embed = nn.Embedding.from_pretrained(init_weights)
        self.embed = nn.Embedding(self.vocab_size, embed_dim)
        self.dropout = nn.Dropout(0.5)

        
    def forward(self, image_features, image_captions):
        image_features = image_features.unsqueeze(1)
        embeddings = self.dropout(self.embed(image_captions))
        #print(embeddings.shape, image_features.shape)
        embeddings = torch.cat((image_features, embeddings[:,:-1]), dim = 1)
        #embeddings = torch.cat((image_features, embeddings), dim = 1)
        hiddens, _ = self.lstm(embeddings)
        outputs = self.linear(hiddens)
        
        return outputs

In [12]:
units = 512
class ImageCaptionsNet(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
        super(ImageCaptionsNet, self).__init__()              
        self.Encoder = Encoder(embed_dim = embedding_dim)
        self.Decoder = Decoder(embedding_dim, hidden_size, vocab_size, num_layers)    
        

    def forward(self, img_batch, cap_batch):
        x = self.Encoder(img_batch)
        #x = x.long().numpy()
        #print(x.shape, "hihi")
        out = self.Decoder(x, cap_batch)
        return out
    def caption_image(self, image, cap_obj, max_length = 10):
        result_cap = []
        #print("initial img ", image.shape)
        with torch.no_grad():
            x = self.Encoder(image)
            print(x.shape)
            x = x.unsqueeze(0)
            #x = self.Encoder(image)
            states = None

            for _ in range(max_length):
                #print("X new = ", x.shape)
                hiddens, states = self.Decoder.lstm(x, states)
                #print("Hiddens = ", hiddens.shape)
                output = self.Decoder.linear(hiddens.squeeze(0))
                predicted = output.argmax(1)
                #predicted = output.argmax(3)
                #print("OT, PD", output.shape, predicted.shape)
                result_cap.append(predicted.item())
                x = self.Decoder.embed(predicted).unsqueeze(0)

                if (cap_obj.index_to_word[predicted.item()] == '[END]'):
                    break
        return [cap_obj.index_to_word[int(idx)] for idx in result_cap]
device = "cuda" if torch.cuda.is_available() else "cpu"

In [13]:
# os.chdir('/content/drive/MyDrive/data/train_data_main/')
# IMAGE_DIR = '/content/drive/MyDrive/data/train_data_main/'

IMAGE_DIR = '/Users/pratyushsaini/Documents/Semester 5/COL774/Assignment-4'
embed_size = 256
hidden_size = 256
num_layers = 1
vocab_size = len(captions_preprocessing_obj.vocab)
net = ImageCaptionsNet(embed_size, hidden_size, vocab_size, num_layers)

#net = net.to(torch.device("cuda:0"))
net = net.to(torch.device("cpu"))
# Creating the Dataset
train_dataset = ImageCaptionsDataset(
    IMAGE_DIR, captions_preprocessing_obj.captions_dict, img_transform=img_transform,
    captions_transform=captions_preprocessing_obj.captions_transform
)
print("Train Dataset loaded")
# Define your hyperparameters
NUMBER_OF_EPOCHS = 1
LEARNING_RATE = 1e-1
BATCH_SIZE = 8
NUM_WORKERS = 0 # Parallel threads for dataloading
print
loss_function = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.SGD(list(net.Decoder.parameters()) + list(net.Encoder.parameters()), lr=LEARNING_RATE)
print("Optimizer loaded")
# Creating the DataLoader for batching purposes
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, collate_fn = collate_fn)
print("Train Loader loaded")
torch.backends.cudnn.benchmark = True
start = time()
loss_list = []
import os
for epoch in range(NUMBER_OF_EPOCHS):
    print("Epoch {}".format(epoch+1))
    iteration = 0
    for batch_idx, sample in enumerate(train_loader):
        net.Encoder.zero_grad()
        net.Decoder.zero_grad()
        optimizer.zero_grad()
        image_batch, captions_batch = sample['image'], sample['captions']

        #If GPU training required
        image_batch = image_batch.float()
        #captions_batch = captions_batch.float()
        #image_batch, captions_batch = image_batch.cuda(), captions_batch.cuda()
        try:
            output_captions = net(image_batch, captions_batch)
        except:
            print("---Error {}".format(batch_idx))
            continue
        #print(output_captions.shape, captions_batch.shape)
        loss = loss_function(output_captions.reshape(-1, output_captions.shape[2]), captions_batch.reshape(-1))
        loss_list.append(loss.item())
        loss.backward()
        optimizer.step()
        print("Iteration: {}, Loss: {}, TimeElapsed: {}Min".format(iteration+1, round(loss.item(), 2), round((time()-start)/60,2), ))
        iteration+=1
        if (iteration > 1):
            break

VOCAB SIZE =  7356
Train Dataset loaded
Optimizer loaded
Train Loader loaded
Epoch 1
torch.Size([8, 256]) hihi
Iteration: 1, Loss: 8.93, TimeElapsed: 0.11Min
torch.Size([8, 256]) hihi
Iteration: 2, Loss: 8.93, TimeElapsed: 0.15Min


## Prediction

In [15]:
class TestDatasetLoader(Dataset):
    
    def __init__(self, img_dir, img_transform):
        """
        Args:
            img_dir (string): Directory with all the test images.            
            img_transform (callable, optional): Optional transform to be applied
                on the image sample.
        """
        self.img_dir = img_dir
        self.img_transform = img_transform
        
        self.image_ids = ['test_data/test' + str(i) + '.jpg' for i in range(1, 5001)]
        
    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_name = self.image_ids[idx]
        image = io.imread(img_name)
        
        if self.img_transform:
            image = self.img_transform(image)
        angle_in_degrees = 45

        #output = torch.from_numpy(ndimage.rotate(alpha, angle_in_degrees, reshape=False))
        # sample = {
        #     'top': image,
        #     'left': torch.from_numpy(ndimage.rotate(image, 90, reshape=False)),
        #     'bottom': torch.from_numpy(ndimage.rotate(image, 180, reshape=False)),
        #     'right': torch.from_numpy(ndimage.rotate(image, 270, reshape=False))
        #     }
        sample['image'] = image # 3* 256 * 256
        return sample

In [16]:
TEST_IMAGE_DIR = '/Users/pratyushsaini/Documents/Semester 5/COL774/Assignment-4'

test_img_transform = transforms.Compose([Rescale(IMAGE_RESIZE), ToTensor(), transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) ]) # Applied sequentially

# Creating the Dataset
test_dataset = TestDatasetLoader(TEST_IMAGE_DIR, img_transform=test_img_transform)

test_loader  = DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers=NUM_WORKERS)

#output_caption = net.predict(device, test_loader)

In [17]:
pred_caps = {}
for batch_idx, sample in enumerate(test_loader):
        #print(, batch_idx)
        image = sample['image']
        #print("ld", image.shape)
        image = image.float()
        #print("Lolxd" , image.shape)
        caption_pred = net.caption_image(image, captions_preprocessing_obj, max_length = 10)
        #print(np.asarray(caption_pred).shape)
        caption_pred = " ".join(caption_pred)
        cap = caption_pred.replace("[START]","").replace("[END]","")
        print("Image_idx ", batch_idx,": ", caption_pred)
        #print("Predicted",batch_idx, pred_cap)

initial img  torch.Size([1, 3, 256, 256])


[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.


torch.Size([1, 256])
X new =  torch.Size([1, 1, 256])
OT, PD torch.Size([1, 7356]) torch.Size([1])
X new =  torch.Size([1, 1, 256])
OT, PD torch.Size([1, 7356]) torch.Size([1])
X new =  torch.Size([1, 1, 256])
OT, PD torch.Size([1, 7356]) torch.Size([1])
X new =  torch.Size([1, 1, 256])
OT, PD torch.Size([1, 7356]) torch.Size([1])
X new =  torch.Size([1, 1, 256])
OT, PD torch.Size([1, 7356]) torch.Size([1])
X new =  torch.Size([1, 1, 256])
OT, PD torch.Size([1, 7356]) torch.Size([1])
X new =  torch.Size([1, 1, 256])
OT, PD torch.Size([1, 7356]) torch.Size([1])
X new =  torch.Size([1, 1, 256])
OT, PD torch.Size([1, 7356]) torch.Size([1])
X new =  torch.Size([1, 1, 256])
OT, PD torch.Size([1, 7356]) torch.Size([1])
X new =  torch.Size([1, 1, 256])
OT, PD torch.Size([1, 7356]) torch.Size([1])
Image_idx  0 :  inter swimsuit woamn house perched enact trunk vocalist crutches guy
initial img  torch.Size([1, 3, 256, 256])
torch.Size([1, 256])
X new =  torch.Size([1, 1, 256])
OT, PD torch.Size(

KeyboardInterrupt: 